## License Plate Detection using Fuzzy Join 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA server

We are reusing the start server notebook for launching the EVA server.

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File ‘00-start-eva-server.ipynb’ already there; not retrieving.



Note: you may need to restart the kernel to use updated packages.
nohup eva_server > eva.log 2>&1 &


Note: you may need to restart the kernel to use updated packages.


### Adding the images to EVADB for analysis

In [2]:
cursor.execute('DROP TABLE IF EXISTS MyImages;')
response = cursor.fetch_all()
response.as_df()

# Download images
!wget -nc "https://www.dropbox.com/s/770stddqfl0psog/license.zip"

!unzip -n license.zip

cursor.execute('LOAD IMAGE "license/Cars0.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars1.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars2.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars3.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars4.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars5.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('LOAD IMAGE "license/Cars6.png" INTO MyImages;')
response = cursor.fetch_all()
response.as_df()

--2023-05-12 08:47:00--  https://www.dropbox.com/s/770stddqfl0psog/license.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 

302 Found
Location: /s/raw/770stddqfl0psog/license.zip [following]
--2023-05-12 08:47:00--  https://www.dropbox.com/s/raw/770stddqfl0psog/license.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 

302 Found
Location: https://uccf55cc8e98be0ee5f099f88caa.dl.dropboxusercontent.com/cd/0/inline/B75m3rE6SYUznra7NClRKkdsBlodZyJABLDnMfiEL1WAa4fuNMTQRFzyg7HkW2vnsB03cwwRAdiFy_tf1nqgKCgQXbcOF9QUSRQqlh92oVNayDN87Vx63D64Y_1QFNuVEUC4AyEJ7Hvx6hpLt2x6R8t3JDYVFPyXkFnj-Y-FuJKhpg/file# [following]
--2023-05-12 08:47:00--  https://uccf55cc8e98be0ee5f099f88caa.dl.dropboxusercontent.com/cd/0/inline/B75m3rE6SYUznra7NClRKkdsBlodZyJABLDnMfiEL1WAa4fuNMTQRFzyg7HkW2vnsB03cwwRAdiFy_tf1nqgKCgQXbcOF9QUSRQqlh92oVNayDN87Vx63D64Y_1QFNuVEUC4AyEJ7Hvx6hpLt2x6R8t3JDYVFPyXkFnj-Y-FuJKhpg/file
Resolving uccf55cc8e98be0ee5f099f88caa.dl.dropboxusercontent.com (uccf55cc8e98be0ee5f099f88caa.dl.dropboxusercontent.com)... 

162.125.9.15, 2620:100:601f:15::a27d:90f
Connecting to uccf55cc8e98be0ee5f099f88caa.dl.dropboxusercontent.com (uccf55cc8e98be0ee5f099f88caa.dl.dropboxusercontent.com)|162.125.9.15|:443... connected.
HTTP request sent, awaiting response... 

302 Found
Location: /cd/0/inline2/B77D-WK4QsCOSPgdNDGLI9nwf_EqyJUwAHGpDuBQUgQRx7mQ5u_9o-ug4x7SbPzHqi22r47ga11JXv-NL7Vm_UjSiSRTJTTNPD0cJtc4u1uL3kMJZtiTtekIUw4sFC8OPm5RwBTsXR0W_6qqzVqd_bK8XaRj8vgnv-jAbybOEhOk000d5SbCoKtAGxcD9HEICD1C63OV6xh9mOqqfAh0tLOx5EV1pcMC2gjpMv5G2DuxjKgB9ORn3csvDiUuHwUzuDdOo9CM6c2h1Vl2N1oxgO4FbyeOTc6ji7D-vE1LUCT4ZyfYVjco4MB-PYw6j3t7CnDRTfi6Y2l_r52O-3N1pedpT1UrlnqaJs2qYgtkp7QMpp6WW8TynsDhv3Acnpc-pZlxuSX9ktezok0qGhF_9lOy2DyYZD4XDiKeONsF5iQk2A/file [following]
--2023-05-12 08:47:00--  https://uccf55cc8e98be0ee5f099f88caa.dl.dropboxusercontent.com/cd/0/inline2/B77D-WK4QsCOSPgdNDGLI9nwf_EqyJUwAHGpDuBQUgQRx7mQ5u_9o-ug4x7SbPzHqi22r47ga11JXv-NL7Vm_UjSiSRTJTTNPD0cJtc4u1uL3kMJZtiTtekIUw4sFC8OPm5RwBTsXR0W_6qqzVqd_bK8XaRj8vgnv-jAbybOEhOk000d5SbCoKtAGxcD9HEICD1C63OV6xh9mOqqfAh0tLOx5EV1pcMC2gjpMv5G2DuxjKgB9ORn3csvDiUuHwUzuDdOo9CM6c2h1Vl2N1oxgO4FbyeOTc6ji7D-vE1LUCT4ZyfYVjco4MB-PYw6j3t7CnDRTfi6Y2l_r52O-3N1pedpT1UrlnqaJs2qYgtkp7QMpp6WW8TynsDhv3Acnpc-pZlxuSX9ktezok0qGhF_9lOy2DyYZD4XD

200 OK
Length: 3281767 (3.1M) [application/zip]
Saving to: ‘license.zip’

license.zip         100%[===================>]   3.13M  --.-KB/s    in 0.02s   

2023-05-12 08:47:01 (162 MB/s) - ‘license.zip’ saved [3281767/3281767]



Archive:  license.zip
   creating: license/
  inflating: license/Cars3.png       
  inflating: __MACOSX/license/._Cars3.png  
  inflating: license/Cars2.png       
  inflating: __MACOSX/license/._Cars2.png  
  inflating: license/Cars0.png       
  inflating: __MACOSX/license/._Cars0.png  
  inflating: license/Cars1.png       
  inflating: __MACOSX/license/._Cars1.png  
  inflating: license/Cars5.png       
  inflating: __MACOSX/license/._Cars5.png  
  inflating: license/Cars4.png       
  inflating: __MACOSX/license/._Cars4.png  
  inflating: license/Cars6.png       
  inflating: __MACOSX/license/._Cars6.png  


,0
0,Number of loaded IMAGE: 1


### License Plate Recognition

In [3]:
# Download UDFS

!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/ndarray/fuzzy_join.py

!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/ocr_extractor.py

cursor.execute("""DROP UDF IF EXISTS OCRExtractor;""")
response = cursor.fetch_all()
response.as_df()

cursor.execute("""DROP UDF IF EXISTS FuzzDistance;""")
response = cursor.fetch_all()
response.as_df()

cursor.execute("""CREATE UDF IF NOT EXISTS OCRExtractor
                  INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
                  OUTPUT (labels NDARRAY STR(10),
                          bboxes NDARRAY FLOAT32(ANYDIM, 4),
                          scores NDARRAY FLOAT32(ANYDIM))
                  TYPE  OCRExtraction
                  IMPL  'ocr_extractor.py' ;
      """)
response = cursor.fetch_all()
response.as_df()

cursor.execute("""CREATE UDF IF NOT EXISTS FuzzDistance
                    INPUT (Input_Array1 NDARRAY ANYTYPE, Input_Array2 NDARRAY ANYTYPE)
                    OUTPUT (distance FLOAT(32, 7))
                    TYPE NdarrayUDF
                    IMPL "fuzzy_join.py";
                    """)
response = cursor.fetch_all()
response.as_df()

--2023-05-12 08:47:03--  https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/ndarray/fuzzy_join.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 1179 (1.2K) [text/plain]
Saving to: ‘fuzzy_join.py’

fuzzy_join.py       100%[===================>]   1.15K  --.-KB/s    in 0s      

2023-05-12 08:47:03 (33.0 MB/s) - ‘fuzzy_join.py’ saved [1179/1179]



--2023-05-12 08:47:03--  https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/ocr_extractor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.


HTTP request sent, awaiting response... 

200 OK
Length: 2750 (2.7K) [text/plain]
Saving to: ‘ocr_extractor.py’

ocr_extractor.py    100%[===================>]   2.69K  --.-KB/s    in 0s      

2023-05-12 08:47:04 (26.0 MB/s) - ‘ocr_extractor.py’ saved [2750/2750]



,0
0,UDF FuzzDistance successfully added to the dat...


In [4]:
cursor.execute("CREATE TABLE IF NOT EXISTS LicensePlateCSV(id INTEGER UNIQUE, label TEXT(30));")
response = cursor.fetch_all()
response.as_df()

""


In [5]:
cursor.execute("LOAD CSV 'data.csv' INTO LicensePlateCSV;")
response = cursor.fetch_all()
response.as_df()


,CSV,Number of loaded frames
0,data.csv,5


In [6]:
cursor.execute('SELECT OCRExtractor(data) FROM MyImages;')
response = cursor.fetch_all()
response.as_df()

,ocrextractor.labels,ocrextractor.bboxes,ocrextractor.scores
0,[KLo1CN2555],"[[[240, 115], [425, 115], [425, 178], [240, 17...",[0.20029704378116528]
1,"[Coi, ZZm, PGoNMN112]","[[[320, 96], [342, 96], [342, 104], [320, 104]...","[0.2585151841538518, 0.009615490492723363, 0.1..."
2,[PReINLR],"[[[227, 173], [269, 173], [269, 193], [227, 19...",[0.12317288475094909]
3,[DZI7 YXR],"[[[150, 126], [250, 126], [250, 158], [150, 15...",[0.6978633091234704]
4,"[pPuie, BES]","[[[175.83355283634924, 85.05243648347647], [38...","[0.19272780155017014, 0.9999165839414489]"
5,"[Wet, alamy stock photo]","[[[362, 274], [384, 274], [384, 280], [362, 28...","[0.002928912305910474, 0.9390591079980826]"
6,"[802.LIN, MMay, VIRGINIA, 07]","[[[157, 141], [370, 141], [370, 219], [157, 21...","[0.8629838375738593, 0.3385108709335327, 0.999..."


In [7]:
cursor.execute('SELECT OCRExtractor(data) FROM MyImages;')
response = cursor.fetch_all()
response.as_df()

cursor.execute('SELECT * FROM LicensePlateCSV;')
response = cursor.fetch_all()
response.as_df()

,licenseplatecsv._row_id,licenseplatecsv.id,licenseplatecsv.label
0,1,1,KLG1CA2555
1,2,2,PGMN112
2,3,3,PRENUP
3,4,4,DZ17YXR
4,5,5,PUI8BES
5,6,1,KLG1CA2555
6,7,2,PGMN112
7,8,3,PRENUP
8,9,4,DZ17YXR
9,10,5,PUI8BES


### Run Fuzzy Join to match Detected License Plate against Local License Plate Database (csv)

In [8]:
cursor.execute("""
   SELECT * FROM MyImages 
       JOIN LATERAL OCRExtractor(data) AS T(a,b,c) 
       JOIN LicensePlateCSV B 
       ON FuzzDistance(T.a, B.label) > 50;
       """)
response = cursor.fetch_all()
response.as_df()

,myimages._row_id,myimages.name,myimages.data,B._row_id,B.id,B.label,T.a,T.b,T.c
0,1,license/Cars0.png,"[[[25, 75, 100], [73, 130, 159], [52, 127, 158...",1,1,KLG1CA2555,[KLo1CN2555],"[[[240, 115], [425, 115], [425, 178], [240, 17...",[0.20029704378116528]
